# Module 0.1: Taste Demo — Salience Scoring — Core

**Arc 0: Probabilistic Foundations** | Module 1 of 8

**Prerequisites**: None

**Time**: ~60-90 minutes

**Implementation target**: buildlog `SalienceScorer` — replaces substring matching for rule compliance evaluation

---

## Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Explain why substring matching fails for evaluating agent rule compliance
- [ ] Decompose rule compliance into linguistic, structural, and outcome signals
- [ ] Implement a `SalienceScorer` with configurable, updatable weights
- [ ] State the falsifiable claim for your scorer and test it against intuitive ratings

In [ ]:
# Provided Code - Do NOT Edit
import re
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Optional, Callable

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# ═══════════════════════════════════════════════════════════════════════════════
# INTRO
# ═══════════════════════════════════════════════════════════════════════════════

*[Hero image placeholder — a detective inspecting code through a magnifying glass,
but the magnifying glass shows a probability distribution instead of the code]*

## The Setup

You built a rule learning system. It uses a Thompson Sampling bandit to select
which rules to surface to AI coding agents. The bandit needs a reward signal:
did the agent follow the rule?

The current answer: `if rule_text in agent_output`.

That is, we check if the rule's exact text appears somewhere in the output.
This is the engineering equivalent of checking if someone read a book by
searching their essay for the book's title.

It's wrong. Let's build something better.

**By the end of this notebook**, you'll have a working `SalienceScorer` that
evaluates rule compliance through three signals — linguistic, structural, and
outcome — and can explain its scores in plain English.

Let's go.

# ═══════════════════════════════════════════════════════════════════════════════
# LAYER 0: THE PROBLEM
# ═══════════════════════════════════════════════════════════════════════════════

Here are two rules from a real buildlog CLAUDE.md:

- *"Always define interfaces before implementations"*
- *"Always validate parsed dates are within valid ranges"*

And here's agent output from a coding session:

```python
class PaymentProcessor:
    def process(self, amount: float) -> bool:
        if amount <= 0:
            raise ValueError("Amount must be positive")
        return self._charge(amount)
```

The `contains` check says: neither rule was followed (the text doesn't appear).

A human reviewer says: the interface rule is irrelevant here (no interface needed
for a standalone processor), and the date rule is irrelevant (no dates involved).
The agent did fine.

The `contains` check is giving the bandit *wrong reward signals*. The bandit is
learning from noise. This is worse than useless — it's actively harmful.

**Three failure modes of substring matching:**

1. **False negatives**: Agent follows the rule's *intent* without quoting it verbatim
2. **False positives**: Agent mentions the rule text without actually following it
3. **Irrelevance blindness**: Can't distinguish "rule violated" from "rule doesn't apply"

# ═══════════════════════════════════════════════════════════════════════════════
# LAYER 1: INTUITION
# ═══════════════════════════════════════════════════════════════════════════════

Think about how a human code reviewer evaluates compliance. They check three things:

1. **Does the output speak the rule's language?** (Linguistic signal)
   - If the rule says "define interfaces before implementations" and the code has
     `Protocol`, `ABC`, `abstractmethod` — that's a vocabulary match.

2. **Does the output follow the rule's prescribed pattern?** (Structural signal)
   - Is there an abstract class defined *before* the concrete class?

3. **Did the task succeed?** (Outcome signal)
   - A rule can't be "followed" if the thing it was supposed to help with broke.

Our scorer combines these three signals with weights:

```
S = w_l * linguistic + w_s * structural + w_o * outcome
```

This is a weighted linear combination. It's the simplest model that could work.
It's definitely wrong in interesting ways (we'll fix that later).
But it's already dramatically better than `contains`.

# ═══════════════════════════════════════════════════════════════════════════════
# LAYER 2: CODE + VIZ
# ═══════════════════════════════════════════════════════════════════════════════

# -------------------------------------------------------------------------------
# Problem 1: Set Up Mock Buildlog Entries and Show How `contains` Fails
# -------------------------------------------------------------------------------

Let's create realistic mock data. Each entry has a rule, agent output, task
outcome, and your intuitive compliance rating (0-1) as ground truth.

In [ ]:
@dataclass
class BuildlogEntry:
    """A single buildlog entry with a rule, agent output, and ground truth."""
    rule: str
    task: str
    agent_output: str
    task_succeeded: bool
    intuitive_compliance: float  # Your expert rating, 0-1


ENTRIES = [
    # Entry 0: Perfect compliance -- interface defined before implementation
    BuildlogEntry(
        rule="Always define interfaces before implementations",
        task="Create a storage backend for user profiles",
        agent_output="""I'll start with the interface:
```python
from abc import ABC, abstractmethod

class StorageBackend(ABC):
    @abstractmethod
    def save(self, key: str, data: dict) -> None: ...
    @abstractmethod
    def load(self, key: str) -> dict: ...

class PostgresBackend(StorageBackend):
    def save(self, key, data):
        self.conn.execute("INSERT INTO profiles ...", data)
    def load(self, key):
        return self.conn.execute("SELECT ...", key).fetchone()
```""",
        task_succeeded=True,
        intuitive_compliance=0.95,
    ),
    # Entry 1: Rule doesn't apply -- no interface needed
    BuildlogEntry(
        rule="Always define interfaces before implementations",
        task="Write a utility function to slugify strings",
        agent_output="""```python
import re

def slugify(text: str) -> str:
    text = text.lower().strip()
    text = re.sub(r'[^\\w\\s-]', '', text)
    return re.sub(r'[-\\s]+', '-', text)
```
Simple utility -- no interface needed here.""",
        task_succeeded=True,
        intuitive_compliance=0.5,
    ),
    # Entry 2: Rule violated -- no interface, but one was needed
    BuildlogEntry(
        rule="Always define interfaces before implementations",
        task="Create a payment processor with multiple gateways",
        agent_output="""```python
class StripeProcessor:
    def charge(self, amount): ...

class PayPalProcessor:
    def charge(self, amount): ...
```
Both processors handle charges directly.""",
        task_succeeded=True,
        intuitive_compliance=0.15,
    ),
    # Entry 3: Perfect date validation
    BuildlogEntry(
        rule="Always validate parsed dates are within valid ranges",
        task="Parse user-submitted event dates",
        agent_output="""```python
from datetime import datetime

def parse_event_date(raw: str) -> datetime:
    dt = datetime.fromisoformat(raw)
    if dt.year < 2020 or dt.year > 2030:
        raise ValueError(f"Date {dt} outside valid range 2020-2030")
    return dt
```""",
        task_succeeded=True,
        intuitive_compliance=0.90,
    ),
    # Entry 4: Date parsing without validation
    BuildlogEntry(
        rule="Always validate parsed dates are within valid ranges",
        task="Import CSV with timestamps",
        agent_output="""```python
import csv
from datetime import datetime

def import_csv(path):
    with open(path) as f:
        for row in csv.reader(f):
            ts = datetime.fromisoformat(row[3])
            yield {"name": row[0], "timestamp": ts}
```""",
        task_succeeded=True,
        intuitive_compliance=0.20,
    ),
    # Entry 5: No dates involved at all
    BuildlogEntry(
        rule="Always validate parsed dates are within valid ranges",
        task="Implement retry logic for HTTP requests",
        agent_output="""```python
import time
import httpx

def retry(url, max_retries=3):
    for i in range(max_retries):
        try:
            return httpx.get(url)
        except httpx.TimeoutException:
            time.sleep(2 ** i)
    raise RuntimeError(f"Failed after {max_retries} retries")
```""",
        task_succeeded=True,
        intuitive_compliance=0.5,
    ),
]

print(f"Loaded {len(ENTRIES)} mock buildlog entries.")
print(f"Rules: {set(e.rule for e in ENTRIES)}")

Now let's see how `contains` does:

In [ ]:
def contains_check(rule: str, output: str) -> float:
    """The current approach: does the rule text appear in the output?"""
    return 1.0 if rule.lower() in output.lower() else 0.0


print("Contains check results vs intuitive ratings:\n")
print(f"{'Entry':>7}  {'Contains':>9}  {'Intuitive':>10}  {'Delta':>6}  Rule")
print("  " + "-" * 70)
for i, e in enumerate(ENTRIES):
    c = contains_check(e.rule, e.agent_output)
    delta = abs(c - e.intuitive_compliance)
    marker = " <<<" if delta > 0.4 else ""
    print(f"  Entry {i}:  {c:>8.2f}  {e.intuitive_compliance:>10.2f}  {delta:>5.2f}{marker}")

print("\n<<< = error > 0.4 (bandit is learning wrong from these)")

Every entry gets 0.0 from `contains` — the rule text never appears verbatim.
The bandit thinks *no rule was ever followed*. That's wrong for entries 0 and 3,
and misleading for 1 and 5 (where the rule doesn't apply).

Let's fix this one signal at a time.

# -------------------------------------------------------------------------------
# Problem 2: Linguistic Signal Detection
# -------------------------------------------------------------------------------

The linguistic signal asks: does the output use vocabulary associated with the rule?

Your task:
1. Implement `linguistic_signal(rule, output)` returning float in [0, 1]
2. Extract keywords from the rule (words > 3 chars)
3. Check for each keyword (or synonyms) in the output
4. Weight code occurrences at 1.0, prose at 0.5

Hint: separate code blocks from prose first.

In [ ]:
# Provided Code - Do NOT Edit
SYNONYM_MAP = {
    "interface": ["interface", "protocol", "abc", "abstract"],
    "interfaces": ["interface", "protocol", "abc", "abstract"],
    "implementations": ["implementation", "concrete", "class"],
    "define": ["define", "create", "class"],
    "validate": ["validate", "check", "verify", "assert", "raise"],
    "parsed": ["parsed", "parse", "fromisoformat", "strptime"],
    "dates": ["date", "datetime", "timestamp"],
    "valid": ["valid", "range", "between", "boundary"],
    "ranges": ["range", "between", "min", "max", "limit"],
    "always": ["always"],
    "within": ["within", "inside", "between"],
    "before": ["before", "first", "prior"],
}


def extract_code_blocks(text: str) -> tuple[str, str]:
    """Split text into (code, prose) by extracting ```...``` blocks."""
    code_blocks = re.findall(r'```(?:python)?\n(.*?)```', text, re.DOTALL)
    code = '\n'.join(code_blocks)
    prose = re.sub(r'```(?:python)?\n.*?```', '', text, flags=re.DOTALL)
    return code, prose

In [ ]:
def linguistic_signal(rule: str, output: str) -> float:
    """
    Measure how much of the rule's key vocabulary appears in the output.
    Code mentions weighted 1.0, prose mentions weighted 0.5.
    Returns float in [0, 1].
    """
    # --- YOUR CODE BELOW ---
    pass


# >>> SOLUTION (collapsed by default)
# ┌─────────────────────────────────────────────────────────────────────────────
# │ def linguistic_signal(rule: str, output: str) -> float:
# │     keywords = [w.lower() for w in rule.split() if len(w) > 3]
# │     if not keywords:
# │         return 0.0
# │     code, prose = extract_code_blocks(output)
# │     code_lower, prose_lower = code.lower(), prose.lower()
# │     scores = []
# │     for kw in keywords:
# │         synonyms = SYNONYM_MAP.get(kw, [kw])
# │         in_code = any(s in code_lower for s in synonyms)
# │         in_prose = any(s in prose_lower for s in synonyms)
# │         if in_code:
# │             scores.append(1.0)
# │         elif in_prose:
# │             scores.append(0.5)
# │         else:
# │             scores.append(0.0)
# │     return min(1.0, sum(scores) / len(scores))
# └─────────────────────────────────────────────────────────────────────────────


# Test
print("Linguistic signal scores:")
for i, e in enumerate(ENTRIES):
    score = linguistic_signal(e.rule, e.agent_output)
    print(f"  Entry {i}: {score:.2f}  (intuitive: {e.intuitive_compliance:.2f})")

# -------------------------------------------------------------------------------
# Problem 3: Structural Signal Detection
# -------------------------------------------------------------------------------

The structural signal asks: does the output's *structure* match the rule's
prescribed pattern? This is rule-specific.

Your task:
1. Implement `check_interface_before_impl(output)` — score in [0, 1]
2. Implement `check_date_validation(output)` — score in [0, 1]
3. Wire them up via `structural_signal(rule, output)`

In [ ]:
def check_interface_before_impl(output: str) -> float:
    """
    1.0 = clear interface-first pattern
    0.5 = interface exists but order unclear
    0.25 = no interface but agent acknowledged rule doesn't apply
    0.0 = no interface pattern detected
    """
    # --- YOUR CODE BELOW ---
    pass


def check_date_validation(output: str) -> float:
    """
    1.0 = date parsing with explicit range validation
    0.5 = some validation but incomplete
    0.25 = dates present but no validation
    0.0 = no date handling detected
    """
    # --- YOUR CODE BELOW ---
    pass


# >>> SOLUTION (collapsed by default)
# ┌─────────────────────────────────────────────────────────────────────────────
# │ def check_interface_before_impl(output: str) -> float:
# │     code, prose = extract_code_blocks(output)
# │     has_abstract = bool(re.search(
# │         r'(class\s+\w+\(\s*(ABC|Protocol)\s*\)|@abstractmethod)', code
# │     ))
# │     has_concrete = bool(re.search(r'class\s+\w+', code))
# │     if has_abstract and has_concrete:
# │         abstract_pos = re.search(
# │             r'(class\s+\w+\(\s*(ABC|Protocol)\s*\)|Protocol)', code
# │         ).start()
# │         for m in re.finditer(r'class\s+\w+', code):
# │             if m.start() != abstract_pos and m.start() > abstract_pos:
# │                 return 1.0
# │         return 0.5
# │     acknowledged = bool(re.search(
# │         r'(no.*interface.*needed|utility|simple)', prose, re.I
# │     ))
# │     if acknowledged:
# │         return 0.25
# │     return 0.0
# │
# │ def check_date_validation(output: str) -> float:
# │     code, _ = extract_code_blocks(output)
# │     has_date = bool(re.search(r'(datetime|fromisoformat|strptime)', code, re.I))
# │     has_range = bool(re.search(
# │         r'(if.*(<|>|<=|>=).*\d|raise.*ValueError)', code, re.I
# │     ))
# │     if has_date and has_range:
# │         return 1.0
# │     if has_date:
# │         return 0.25
# │     return 0.0
# └─────────────────────────────────────────────────────────────────────────────


STRUCTURAL_PATTERNS = {
    "Always define interfaces before implementations": check_interface_before_impl,
    "Always validate parsed dates are within valid ranges": check_date_validation,
}


def structural_signal(rule: str, output: str) -> float:
    """Dispatch to the appropriate structural checker."""
    checker = STRUCTURAL_PATTERNS.get(rule)
    return checker(output) if checker else 0.5


# Test
print("Structural signal scores:")
for i, e in enumerate(ENTRIES):
    score = structural_signal(e.rule, e.agent_output)
    print(f"  Entry {i}: {score:.2f}  (intuitive: {e.intuitive_compliance:.2f})")

# -------------------------------------------------------------------------------
# Problem 4: Outcome Signal
# -------------------------------------------------------------------------------

The simplest signal: did the task succeed? Binary for now.

In [ ]:
def outcome_signal(entry: BuildlogEntry) -> float:
    """1.0 if task succeeded, 0.0 otherwise."""
    # --- YOUR CODE BELOW ---
    pass


# >>> SOLUTION (collapsed by default)
# ┌─────────────────────────────────────────────────────────────────────────────
# │ def outcome_signal(entry: BuildlogEntry) -> float:
# │     return 1.0 if entry.task_succeeded else 0.0
# └─────────────────────────────────────────────────────────────────────────────

# Tests pass. Moving on.
print("All entries succeeded (outcome = 1.0 for all).")
print("In real data, this differentiates. For now, linguistic + structural do the work.")

# -------------------------------------------------------------------------------
# Problem 5: Assemble the SalienceScorer
# -------------------------------------------------------------------------------

Combine all three signals into a single class.

Your task:
1. Implement `SalienceScorer` with configurable weights
2. Weights must be updatable (constitutional rule: not hardcoded forever)
3. `score` returns a `SalienceResult` with component breakdown
4. `explain` returns plain English (constitutional rule: explain 0.7)
5. State the **falsifiable claim**: what would make this scorer wrong?

In [ ]:
@dataclass
class SalienceResult:
    """Result with component breakdown."""
    score: float
    linguistic: float
    structural: float
    outcome: float
    weights: dict

    def explain(self) -> str:
        """Plain-English explanation."""
        # --- YOUR CODE BELOW ---
        pass


class SalienceScorer:
    """
    Scores agent output for rule compliance using three signals.

    Falsifiable claim: Rankings agree with expert intuitive ratings
    (Spearman rho > 0.8). If not, recalibrate.
    """

    def __init__(self, w_linguistic=0.4, w_structural=0.4, w_outcome=0.2):
        # --- YOUR CODE BELOW ---
        pass

    def update_weights(self, w_l: float, w_s: float, w_o: float) -> None:
        """Update weights. Must sum to 1."""
        # --- YOUR CODE BELOW ---
        pass

    def score(self, entry: BuildlogEntry) -> SalienceResult:
        """Score a single buildlog entry."""
        # --- YOUR CODE BELOW ---
        pass


# >>> SOLUTION (collapsed by default)
# ┌─────────────────────────────────────────────────────────────────────────────
# │ @dataclass
# │ class SalienceResult:
# │     score: float
# │     linguistic: float
# │     structural: float
# │     outcome: float
# │     weights: dict
# │
# │     def explain(self) -> str:
# │         level = (
# │             "strong" if self.score >= 0.75
# │             else "moderate" if self.score >= 0.5
# │             else "weak" if self.score >= 0.25
# │             else "minimal"
# │         )
# │         parts = []
# │         if self.linguistic >= 0.7:
# │             parts.append("uses relevant vocabulary")
# │         elif self.linguistic >= 0.4:
# │             parts.append("some vocabulary overlap")
# │         else:
# │             parts.append("little vocabulary match")
# │         if self.structural >= 0.7:
# │             parts.append("follows the prescribed pattern")
# │         elif self.structural >= 0.4:
# │             parts.append("partially follows the pattern")
# │         else:
# │             parts.append("doesn't follow the pattern")
# │         parts.append("task succeeded" if self.outcome >= 0.5 else "task failed")
# │         return f"Score {self.score:.2f} -- {level} compliance. The output {', '.join(parts)}."
# │
# │ class SalienceScorer:
# │     FALSIFIABLE_CLAIM = "Rankings agree with expert ratings (Spearman rho > 0.8)."
# │
# │     def __init__(self, w_linguistic=0.4, w_structural=0.4, w_outcome=0.2):
# │         self.update_weights(w_linguistic, w_structural, w_outcome)
# │
# │     def update_weights(self, w_l, w_s, w_o):
# │         total = w_l + w_s + w_o
# │         assert abs(total - 1.0) < 1e-6, f"Weights must sum to 1, got {total}"
# │         self.weights = {"linguistic": w_l, "structural": w_s, "outcome": w_o}
# │
# │     def score(self, entry):
# │         l = linguistic_signal(entry.rule, entry.agent_output)
# │         s = structural_signal(entry.rule, entry.agent_output)
# │         o = outcome_signal(entry)
# │         combined = self.weights["linguistic"]*l + self.weights["structural"]*s + self.weights["outcome"]*o
# │         return SalienceResult(round(combined, 4), round(l, 4), round(s, 4), round(o, 4), dict(self.weights))
# └─────────────────────────────────────────────────────────────────────────────

# -------------------------------------------------------------------------------
# Problem 6: Run the Scorer, Visualize, Test the Falsifiable Claim
# -------------------------------------------------------------------------------

Your task:
1. Score all entries with default weights
2. Create a side-by-side bar chart: salience score vs. intuitive rating
3. Compute Spearman rank correlation -- does it beat 0.8?
4. Print the explanation for each entry

In [ ]:
from scipy import stats as scipy_stats

scorer = SalienceScorer(w_linguistic=0.4, w_structural=0.4, w_outcome=0.2)

# --- YOUR CODE BELOW ---
# Score all entries, plot comparison, compute Spearman correlation


# >>> SOLUTION (collapsed by default)
# ┌─────────────────────────────────────────────────────────────────────────────
# │ results = [scorer.score(e) for e in ENTRIES]
# │ salience_scores = [r.score for r in results]
# │ intuitive = [e.intuitive_compliance for e in ENTRIES]
# │
# │ rho, pval = scipy_stats.spearmanr(salience_scores, intuitive)
# │ print(f"Spearman rho: {rho:.3f}  (p={pval:.4f})")
# │ print(f"Falsifiable claim threshold: rho > 0.8")
# │ print(f"Result: {'PASS' if rho > 0.8 else 'NEEDS RECALIBRATION'}")
# │ print()
# │
# │ x = np.arange(len(ENTRIES))
# │ width = 0.35
# │ fig, ax = plt.subplots(figsize=(12, 5))
# │ ax.bar(x - width/2, salience_scores, width, label='Salience Score', color='#2196F3')
# │ ax.bar(x + width/2, intuitive, width, label='Intuitive Rating', color='#FF9800')
# │ ax.set_xlabel('Entry')
# │ ax.set_ylabel('Score')
# │ ax.set_title('SalienceScorer vs. Intuitive Ratings')
# │ ax.set_xticks(x)
# │ ax.legend()
# │ ax.set_ylim(0, 1.1)
# │ plt.tight_layout()
# │ plt.show()
# │
# │ print("\nExplanations:")
# │ for i, (e, r) in enumerate(zip(ENTRIES, results)):
# │     print(f"\n  Entry {i}: {e.task[:50]}")
# │     print(f"    {r.explain()}")
# └─────────────────────────────────────────────────────────────────────────────

## Theory Backfill: Why a Weighted Linear Combination?

We made two assumptions that are both wrong and both useful:

1. **Linearity**: Signals combine additively. In reality, high structural + high
   linguistic is stronger evidence than either alone (interaction effects).
   But linear is the simplest baseline.

2. **Independence**: The three signals are independent. They're not -- vocabulary
   overlap correlates with structural compliance. But treating them as independent
   lets us build and test each detector separately.

Both assumptions break down. That's fine. Module 0.2 builds the probability
foundations to handle more sophisticated models.

# ═══════════════════════════════════════════════════════════════════════════════
# EXERCISES
# ═══════════════════════════════════════════════════════════════════════════════

## Exercise 1: Implement the SalienceScorer

If you used the solutions above, go back and implement them yourself. Verify:
- [ ] Weights are stored and updatable via `update_weights`
- [ ] `update_weights` validates weights sum to 1
- [ ] `score` returns a `SalienceResult` with all components
- [ ] `explain` produces a readable sentence for any score

## Exercise 2: Compare and Tune

Try different weight configurations:
- `(0.7, 0.2, 0.1)` — linguistic-heavy
- `(0.2, 0.7, 0.1)` — structural-heavy
- `(0.33, 0.33, 0.34)` — uniform

Which gives the highest Spearman correlation? Why?

## Exercise 3 [PUBLISH]: Write the "Contains Check Takedown"

Write 500-800 words explaining why substring matching fails for evaluating
agent rule compliance. Target: practitioners building AI agent systems.

Structure:
1. The problem: you have rules, you need to evaluate compliance
2. The naive approach: `contains` / substring matching
3. Three failure modes (use examples from this notebook)
4. The alternative: decompose into linguistic + structural + outcome signals
5. Why this matters for bandit-based rule learning systems

Draft for: *"Here's what everyone gets wrong about evaluating AI agent output"*

In [ ]:
# Exercise 3 workspace
draft = """
# Here's What Everyone Gets Wrong About Evaluating AI Agent Output

TODO: Write your draft here.
"""
print(draft)

# ═══════════════════════════════════════════════════════════════════════════════
# OUTRO
# ═══════════════════════════════════════════════════════════════════════════════

## What Just Happened

You built a salience scorer that replaces blind substring matching with three
targeted signals:

- **Linguistic**: Does the output speak the rule's language?
- **Structural**: Does the output follow the rule's prescribed pattern?
- **Outcome**: Did the task succeed?

Combined with configurable weights (`S = w_l * linguistic + w_s * structural +
w_o * outcome`), this gives the bandit a dramatically better reward signal.

You also established three constitutional rules for the entire arc:

1. Every scoring function must have a falsifiable claim
2. Weights must be updatable from data, not hardcoded forever
3. If you can't explain what a score of 0.7 means in plain English, the scorer isn't ready

## Publication Note

Exercise 3 is a draft for *"Here's what everyone gets wrong about evaluating
AI agent output."* Run an edit pass and it's ready to publish.

## What's Next

The weights are hand-picked. Why 0.4/0.4/0.2? Because it felt right. That's
not science. In Module 0.2, we build the probability foundations to make this
rigorous — so the weights can be learned from data instead of vibes.

--> [Module 0.2: Probability & Counting](../module-0.2-probability-counting/0.2-probability-counting-core.ipynb)

# ═══════════════════════════════════════════════════════════════════════════════
# RESOURCES
# ═══════════════════════════════════════════════════════════════════════════════

- **Salience (cognitive science)**: Salience as attention-weighted relevance comes
  from cognitive/perceptual psychology. Our scorer approximates what a human
  reviewer does intuitively.
- **Weighted linear combinations**: Any introductory linear algebra text covers
  why this is the simplest useful model. We'll formalize this in Arc 1.
- **From the archive**: `archive/v1-week-based/notebooks/` — earlier explorations
  of the bandit that this module's scorer feeds into.
- **Blitzstein & Hwang, *Introduction to Probability***: Ch. 1-2 set up the
  probability framework we'll use starting in Module 0.2.